### OSMnx contributes five primary capabilities  
+ enables automated and on-demand downloading of political boundary geometries  
+ can automate and customize the downloading of street networks from OpenStreetMap and construct them into multidigraphs  
+ can correct and simplify network topology  
+ can save/load street networds to/from disk in various file formats  
+ has built-in functions to analyze street networks, calculate routs, project and visualize networks, and quickly and consistently calculate various metric and topological measures  

In [10]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
import sklearn
%matplotlib inline
ox.config(log_console=True)
ox.__version__
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)

C:\Users\jmccall\AppData\Local\anaconda3\envs\openstreets\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [14]:
#pipwin install gdal

In [15]:
rides = pd.read_csv('../data/accessrides.csv')

In [16]:
rides.head()

,Type,Client ID,Time,Date,Time Stamp,LAT,LONG
0,Pickup,78,15:00:00,"Saturday, January 1, 2022",1/1/2022 15:00,36.145059,-86.812360
1,Pickup,78,18:45:00,"Saturday, January 1, 2022",1/1/2022 18:45,36.194334,-86.618303
2,Pickup,7096,7:00:00,"Saturday, January 1, 2022",1/1/2022 7:00,36.143799,-86.752940
3,Pickup,9607,14:15:00,"Saturday, January 1, 2022",1/1/2022 14:15,36.215131,-86.600926
4,Pickup,9988,12:45:00,"Saturday, January 1, 2022",1/1/2022 12:45,36.145059,-86.812360


In [17]:
gdf = gpd.GeoDataFrame(rides, geometry=gpd.points_from_xy(rides.LONG, rides.LAT), crs = 'NAD83')

In [18]:
gdf['LAT'] = gdf['LAT'].astype(str)
gdf['LONG'] = gdf['LONG'].astype(str)

In [25]:
gdf.tail()

,Type,Client ID,Time,Date,Time Stamp,LAT,LONG,geometry,Points,LatLongPoints
491117,Dropoff,27371,5:30:00,"Saturday, December 31, 2022",12/31/2022 5:30,36.138703,-86.801373,POINT (-86.80137 36.13870),"(-86.801373, 36.138703)","(36.138703, -86.801373)"
491118,Dropoff,27373,6:15:00,"Saturday, December 31, 2022",12/31/2022 6:15,36.162768,-86.679763,POINT (-86.67976 36.16277),"(-86.679763, 36.162768)","(36.162768, -86.679763)"
491119,Dropoff,27373,12:00:00,"Saturday, December 31, 2022",12/31/2022 12:00,36.248777,-86.756963,POINT (-86.75696 36.24878),"(-86.756963, 36.248777)","(36.248777, -86.756963)"
491120,Dropoff,27426,6:45:00,"Saturday, December 31, 2022",12/31/2022 6:45,36.232634,-86.755149,POINT (-86.75515 36.23263),"(-86.755149, 36.232634)","(36.232634, -86.755149)"
491121,Dropoff,27426,11:15:00,"Saturday, December 31, 2022",12/31/2022 11:15,36.302074,-86.737109,POINT (-86.73711 36.30207),"(-86.737109, 36.302074)","(36.302074, -86.737109)"


In [26]:
gdf['Points'] = '(' + gdf['LONG'] + ', ' + gdf['LAT'] + ')'
gdf['LatLongPoints'] = '(' + gdf['LAT'] + ', ' + gdf['LONG'] + ')'

In [31]:
points = list(gdf['LatLongPoints'].unique())

In [27]:
place = 'Nashville, TN, USA'
network_type = 'walk'
trip_times = [5, 10, 15, 20, 25]
travel_speed = 4.5

In [28]:
# download the street network
G = ox.graph_from_place(place, network_type=network_type)

In [29]:
import folium

In [32]:
map = folium.Map((points[0][1],points[0][0]), zoom_start=13)
for pt in points:
    marker = folium.Marker([pt[1], pt[0]]) #latitude,longitude
    map.add_child(marker) 
map

ValueError: Location should consist of two numerical values, but '(' of type <class 'str'> is not convertible to float.

In [37]:
# change to latitude, longitude order
itin = [(pt[1],pt[0]) for i in points] #latitude longitude
# create graph
G = ox.graph_from_point(itin[0],network_type='walk')
# create the path
path = []
for first, second in zip(itin, itin[1:]):
    one = ox.get_nearest_node(G, first)
    two= ox.get_nearest_node(G, second)
    path.append(nx.shortest_path(G,one,two))
print(path)
[[1261505524], [1261505524, 1261505525, 456520278], [456520278, 1261505525, 4237637413, 5549684249], [5549684249, 4237637412], [4237637412, 4237637410], [4237637410, 460438855], [460438855, 456520295], [456520295, 1261505504]]
# plot
fig, ax = ox.plot_graph_routes(G, path,edge_color='k', bgcolor='w')

AttributeError: module 'osmnx' has no attribute 'get_nearest_node'

In [25]:
x, y

(array('d', [-86.75560406332076]), array('d', [36.142829720976906]))

In [15]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [10]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0, return_hex=True)

NameError: name 'trip_times' is not defined

In [20]:
test = ox.geometries.geometries_from_point([36.138703, -86.801373], tags = {'building': True}, dist = 800)

In [21]:
test.head(20)

addr:state    building  ele gnis:county_name gnis:feature_id                      gnis:import_uuid gnis:reviewed                                 name         source                                           geometry amenity                                              nodes               addr:housename addr:housenumber addr:postcode       addr:street                                              image              sport building:levels addr:city operator phone website wheelchair wikidata residential brand brand:wikidata internet_access parking surface tourism access  fee supervised layer roof:levels opening_hours wikipedia man_made tower:type social_facility social_facility:for diet:vegetarian brand:wikipedia office shop addr:country air_conditioning  fax internet_access:fee smoking stars roof:shape leisure cuisine drive_through takeaway short_name bench  bin shelter_type note studio brand:en brand:zh name:en name:zh ways club type operator:type
element_type osmid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
node         368157091          TN         yes  164         Davidson         1954475  57871b70-0100-4405-bb30-88b2e001a944            no  Wilkerson Hearing and Speech Center  USGS Geonames                         POINT (-86.79722 36.14422)     NaN                                                NaN                          NaN              NaN           NaN               NaN                                                NaN                NaN             NaN       NaN      NaN   NaN     NaN        NaN      NaN         NaN   NaN            NaN             NaN     NaN     NaN     NaN    NaN  NaN        NaN   NaN         NaN           NaN       NaN      NaN        NaN             NaN                 NaN             NaN             NaN    NaN  NaN          NaN              NaN  NaN                 NaN     NaN   NaN        NaN     NaN     NaN           NaN      NaN        NaN   NaN  NaN          NaN  NaN    NaN      NaN      NaN     NaN     NaN  NaN  NaN  NaN           NaN
             368157131          TN         yes  160         Davidson         2453540  57871b70-0100-4405-bb30-88b2e001a944            no                         Wesley Place  USGS Geonames                         POINT (-86.79870 36.14580)     NaN                                                NaN                          NaN              NaN           NaN               NaN                                                NaN                NaN             NaN       NaN      NaN   NaN     NaN        NaN      NaN         NaN   NaN            NaN             NaN     NaN     NaN     NaN    NaN  NaN        NaN   NaN         NaN           NaN       NaN      NaN        NaN             NaN                 NaN             NaN             NaN    NaN  NaN          NaN              NaN  NaN                 NaN     NaN   NaN        NaN     NaN     NaN           NaN      NaN        NaN   NaN  NaN          NaN  NaN    NaN      NaN      NaN     NaN     NaN  NaN  NaN  NaN           NaN
             368157957          TN         yes  165         Davidson         1954454  57871b

In [13]:
test = test[['name', 'geometry']]

In [14]:
test = test.reset_index(drop = True).dropna()

In [15]:
test.head()

,name,geometry
0,Wilkerson Hearing and Speech Center,POINT (-86.79722 36.14422)
1,Wesley Place,POINT (-86.79870 36.14580)
2,Learned Laboratory,POINT (-86.80088 36.14446)
3,Ingram Hall,POINT (-86.80560 36.13873)
4,Vanderbilt-Ingram Cancer Center,POINT (-86.80274 36.14013)


In [16]:
coord_list = [(x,y) for x,y in zip(test['geometry'].centroid.x , test['geometry'].centroid.y)]

C:\Users\jmccall\AppData\Local\Temp\ipykernel_5900\1714859182.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(test['geometry'].centroid.x , test['geometry'].centroid.y)]
C:\Users\jmccall\AppData\Local\Temp\ipykernel_5900\1714859182.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(test['geometry'].centroid.x , test['geometry'].centroid.y)]


In [17]:
thing = [(36.138703, -86.801373)]

In [18]:
map = folium.Map((coord_list[0][1],coord_list[0][0]), zoom_start=13)
for pt in coord_list:
    marker = folium.Marker([pt[1], pt[0]]) #latitude,longitude
    map.add_child(marker) 
map

In [19]:
test['name'].unique()

array(['Wilkerson Hearing and Speech Center', 'Wesley Place',
       'Learned Laboratory', 'Ingram Hall',
       'Vanderbilt-Ingram Cancer Center', 'Barbara massey hall',
       'Bear house', 'Featheringill / Jacobs Hall', 'Buttrick Hall',
       'Nuclear Magnetic Resonance Facility',
       'Village at Vanderbilt - South Tower',
       'Village at Vanderbilt - North Tower', 'Wyatt Center',
       'Vanderbilt - Village Commercial', 'Blair School of Music',
       'Martha River Ingram Center for the Performing Arts',
       'McTyeire Hall', 'Bryan Building',
       'Moxy Nashville Vanderbilt Area', 'West House', 'North House',
       'South Garage', "Children's Hospital Outpatient Clinics",
       "Children's Hospital at Vanderbilt", 'Vanderbilt Eye Institute',
       'Hill Center', 'Commons Center', 'Central Parking Garage',
       'Medical Center East North Tower',
       'Medical Center East South Tower', 'Dayani Center',
       'Mayborn Building', 'Payne Building', 'East Garage',
  